In [151]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.model_selection import train_test_split # function for splitting data to train and test sets

import nltk
from nltk.corpus import stopwords
from nltk.classify import SklearnClassifier

import matplotlib.pyplot as plt
%matplotlib inline

from itertools import product
from collections import defaultdict
from IPython.display import HTML
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_curve

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
import glob
import requests, re, spacy
nlp = spacy.load('en')

#### i fudged up, I pushed this 50k dataset to github and it truncated it to only 1000 reviews each
#### so i'm pulling these from my local ( "NLP-CAP/aclImdb/Train/neg/*.txt") should i make a new repo?

In [152]:
import os
os.getcwd()

'/Users/hemingway/Desktop/Galvanize/NLP-CAP/aclImdb'

#### creating a list of file names to call/open since the reviews are all different txt files in a folder

In [153]:
pos = glob.glob("Train/pos/*.txt") #globglob
neg = glob.glob("Train/neg/*.txt") #glob
pos[0],neg[0]

('Train/pos/4715_9.txt', 'Train/neg/1821_4.txt')

In [154]:
#'two pos lists: reviews, and scores (scores labelled in filename)'
pos_reviews = []
pos_scores = []
for i in range(len(pos)):
                                    #read files
    x = open(pos[i],'r')
    content = x.read()
    pos_reviews.append(content)
                                    #read scores
    a = pos[i].split('_')
    b = a[1].split('.txt')
    c = [int(s) for s in b if s.isdigit()]
    pos_scores.append(c[0])

In [155]:
#'same same, but different'
neg_reviews = []
neg_scores = []
for i in range(len(neg)):
    
    x = open(neg[i],'r')
    content = x.read()
    neg_reviews.append(content)
    
    a = neg[i].split('_')
    b = a[1].split('.txt')
    c = [int(s) for s in b if s.isdigit()]
    neg_scores.append(c[0])

In [156]:
def clean_line(line):
    '''
    clean_line
    
    input: line (str) - a line from the script
    return: list of cleaned, lemetized tokens
    
    This does all the NLP preprocessing:
        * lower text
        * split on special characters, while removing annotations
        * remove stop words
        * perform lemmetization
        *** MIGHT REMOVE IMPORTANT WORDS IDK MAN
    '''
    line = line.lower()
    line = [x for x in re.findall(r"[\w()']+", line) if x[0] != "(" and x[-1] != ")"]
    s_stop = set(stopwords.words())
    line = ' '.join([word for word in line if word not in s_stop])
    return [token.lemma_ for token in nlp(line)]

In [144]:
#'cleaning reviews and assigning it to a new list'
p = []
for i in range(len(pos)):
    p.append(clean_line(pos_reviews[i]))

In [158]:
n = []
for i in range(len(neg)):
    n.append(clean_line(neg_reviews[i]))

#### joining the lists of lists to feed into a single column in a dataframe; same for scores.

In [184]:
#reviews and scores
r = p + n
s = pos_scores + neg_scores

In [185]:
#binary list
b = []
for x in s:
    if x > 5:
        b.append(1)
    else:
        b.append(0)


#### made a DF with reviews and corresponding scores, also adding binary column for good or bad

In [186]:
Movies = pd.DataFrame({'Reviews':r})
Movies['Scores'] = s
Movies['Binary'] = b

In [187]:
Movies

,Reviews,Scores,Binary
0,"[movie, get, respect, sure, lot, memorable, qu...",9,1
1,"[bizarre, horror, movie, fill, famous, face, s...",8,1
2,"[solid, unremarkable, film, matthau, einstein,...",7,1
3,"[strange, feel, alone, theater, occupy, parent...",8,1
4,"[probably, already, know, 5, additional, episo...",10,1
5,"[see, movie, two, grown, child, although, clev...",8,1
6,"[use, imdb, br, br, give, hefty, vote, favouri...",10,1
7,"[good, film, powerful, message, love, redempti...",10,1
8,"[make, quartet, trio, continue, quality, early...",10,1
9,"[mature, admit, shed, tear, film, mature, resp...",10,1


['movie',
 'get',
 'respect',
 'sure',
 'lot',
 'memorable',
 'quote',
 'list',
 'gem',
 'imagine',
 'movie',
 'joe',
 'piscopo',
 'actually',
 'funny',
 'maureen',
 'stapleton',
 'scene',
 'stealer',
 'moroni',
 'character',
 'absolute',
 'scream',
 'watch',
 'alan',
 'skipper',
 'hale',
 'jr',
 'police',
 'sgt']